In [1]:
import tensorflow as tf

#download MNIST data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
#convert images into float32 data type
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
#flattening 28*28 image type into 784 dimensions
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
#normalizes values ​​between [0, 255] to values ​​between [0, 1]
x_train, x_test = x_train / 255., x_test / 255.
#apply one-hot encoding to label data
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

#mix data using tf.data API and import it in batch form
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(100)
train_data_iter = iter(train_data)

#define the Softmax Regression model using tf.keras.Model
class SoftmaxRegression(tf.keras.Model):
  def __init__(self):
    super(SoftmaxRegression, self).__init__()
    self.softmax_layer = tf.keras.layers.Dense(10,
                                               activation=None,
                                               kernel_initializer='zeros',
                                               bias_initializer='zeros')

  def call(self, x):
    logits = self.softmax_layer(x)

    return tf.nn.softmax(logits)

#define cross-entropy loss function
@tf.function
def cross_entropy_loss(y_pred, y):
  return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_pred), axis=[1]))

#define the gradient descent optimizer for optimization
optimizer = tf.optimizers.SGD(0.5)

#define a function for optimization
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

#define a function that outputs the accuracy of the model
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

#declare the SoftmaxRegression model
SoftmaxRegression_model = SoftmaxRegression()

#optimization is performed with 1000 iterations
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)
  train_step(SoftmaxRegression_model, batch_xs, batch_ys)

print("Accuracy: %f" % compute_accuracy(SoftmaxRegression_model(x_test), y_test))

11493376/11490434 [==============================] - 0s 0us/step
Accuracy: 0.914100
